In [1]:
import pandas as pd
import psycopg2
import bokeh.plotting
import bokeh.models as bmodels
import bokeh.plotting as plt
import bokeh.palettes as bpalettes
import bokeh.transform as btransform
import bokeh.io

import server.model.connection as smc
import server.model.version as smv
import server.model.event as sme
import server.view.util as smu
import server.season.s2019.view.points as sssvp

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
conn = smc.pool.getconn()

In [3]:
sql = '''
    SELECT points.team, matches, points.cargo_pts, points.cargo_pts/matches AS avg_cargo FROM vw_num_matches LEFT JOIN (SELECT team, SUM(successes) * 3 AS cargo_pts FROM vw_measures 
    WHERE task IN ('autoCargo', 'rocketCargo1', 'rocketCargo2', 'rocketCargo3', 'csCargo')
    AND team IN (SELECT team FROM schedules
                 INNER JOIN status ON schedules.event_id=status.event_id 
                 WHERE schedules.event_id=status.event_id)
    GROUP BY team) AS points ON vw_num_matches.team=points.team;
'''
df_temp1 = pd.read_sql(sql, conn)
# df_Cargos = df_temp.set_index('team')
# df_Cargos
df_temp1 = df_temp1.set_index('team')
df_temp1

,matches,cargo_pts,avg_cargo
team,,,
1258,4,84.0,21.0
1318,3,15.0,5.0
1425,4,39.0,9.0
1778,4,33.0,8.0
1899,3,24.0,8.0
2046,4,69.0,17.0
2557,4,36.0,9.0
2906,4,63.0,15.0
2907,3,60.0,20.0


In [4]:
a = df_temp1[df_temp1.index.notnull()]
a = a.sort_values('avg_cargo', ascending=False)

In [5]:
Cargos_cds = bmodels.ColumnDataSource(a)
Cargos_cds.column_names

['matches', 'cargo_pts', 'avg_cargo', 'team']

In [6]:
hplot = plt.figure(x_range=Cargos_cds.data['team'], plot_width=700, plot_height=250, title="Cargos")

hplot.vbar(x='team', top='avg_cargo', width=0.4, color="#2ca02c", source=Cargos_cds)
hplot.xaxis.major_label_orientation = 3.14/4

bokeh.io.show(hplot)

In [7]:
sql = '''
    SELECT points.team, matches, points.hatch_pts, points.hatch_pts/matches AS avg_hatch FROM vw_num_matches LEFT JOIN (SELECT team, SUM(successes) * 2 AS hatch_pts FROM vw_measures 
    WHERE task IN ('autoHatch', 'rocketHatch1', 'rocketHatch2', 'rocketHatch3', 'csHatch')
    AND team IN (SELECT team FROM schedules
                 INNER JOIN status ON schedules.event_id=status.event_id 
                 WHERE schedules.event_id=status.event_id)
    GROUP BY team) AS points ON vw_num_matches.team=points.team;
'''
df_temp2 = pd.read_sql(sql, conn)
# df_Cargos = df_temp.set_index('team')
# df_Cargos
df_temp2 = df_temp2.set_index('team')
df_temp2

,matches,hatch_pts,avg_hatch
team,,,
1258,4,38,9
1318,3,10,3
1425,4,12,3
1778,4,16,4
1899,3,34,11
2046,4,34,8
2557,4,12,3
2906,4,54,13
2907,3,28,9


In [8]:
a = df_temp2.sort_values('avg_hatch', ascending=False)
Hatches_cds = bmodels.ColumnDataSource(a)
Hatches_cds.column_names

['matches', 'hatch_pts', 'avg_hatch', 'team']

In [9]:
hplot = plt.figure(x_range=Hatches_cds.data['team'], plot_width=700, plot_height=250, title="Hatches")

hplot.vbar(x='team', top='avg_hatch', width=0.4, color="#2ca02c", source=Hatches_cds)
hplot.xaxis.major_label_orientation = 3.14/4

bokeh.io.show(hplot)

In [10]:
sql = '''
    SELECT team, COUNT(CASE WHEN capability='1' THEN 1 END) AS Climb1, COUNT(CASE WHEN capability='2' THEN 1 END) AS Climb2,
    COUNT(CASE WHEN capability='3' THEN 1 END) AS Climb3 FROM vw_measures WHERE task='climb' GROUP BY team;
'''
df_temp3 = pd.read_sql(sql, conn)
# df_Cargos = df_temp.set_index('team')
# df_Cargos
df_temp3 = df_temp3.set_index('team')
df_temp3

,climb1,climb2,climb3
team,,,
1258,4,0,0
1318,1,0,0
1425,1,0,0
1778,2,0,0
1899,0,1,1
2046,1,2,1
2557,3,0,0
2906,1,1,2
2907,3,0,0


In [11]:
df_temp3['climb_pts'] = (df_temp3.climb1 * 3 + df_temp3.climb2 * 6 + df_temp3.climb3 * 12) / (df_temp3.climb1 + df_temp3.climb2 + df_temp3.climb3)
df_temp3

,climb1,climb2,climb3,climb_pts
team,,,,
1258,4,0,0,3.00
1318,1,0,0,3.00
1425,1,0,0,3.00
1778,2,0,0,3.00
1899,0,1,1,9.00
2046,1,2,1,6.75
2557,3,0,0,3.00
2906,1,1,2,8.25
2907,3,0,0,3.00


In [12]:
a = df_temp3.sort_values('climb_pts', ascending=False)
climb_cds = bmodels.ColumnDataSource(a)
climb_cds.column_names
a

,climb1,climb2,climb3,climb_pts
team,,,,
3219,0,1,2,10.00
1899,0,1,1,9.00
2906,1,1,2,8.25
2046,1,2,1,6.75
3684,0,2,0,6.00
4579,0,2,0,6.00
3574,1,2,0,5.00
2926,1,2,0,5.00
492,1,2,0,5.00


In [13]:
hplot = plt.figure(x_range=Hatches_cds.data['team'], plot_width=700, plot_height=250, title="Climb")

hplot.vbar(x='team', top='climb_pts', width=0.4, color="#2ca02c", source=climb_cds)
hplot.xaxis.major_label_orientation = 3.14/4

bokeh.io.show(hplot)

In [14]:
data = pd.DataFrame([df_temp1['avg_cargo'], df_temp2['avg_hatch'], df_temp3['climb_pts']])

data = data.T
data_cds = bmodels.ColumnDataSource(data)
data

,avg_cargo,avg_hatch,climb_pts
1258,21.0,9.0,3.00
1318,5.0,3.0,3.00
1425,9.0,3.0,3.00
1778,8.0,4.0,3.00
1899,8.0,11.0,9.00
2046,17.0,8.0,6.75
2557,9.0,3.0,3.00
2906,15.0,13.0,8.25
2907,20.0,9.0,3.00
2926,10.0,8.0,5.00


In [15]:
points = ["avg_cargo", "avg_hatch", "climb_pts"]
point = ["avg_cargo", "avg_hatch", "climb_pts"]
colors = ["#2ca02c", "#98df8a", "#637939"]
team_list = df_temp2.index.tolist()
team_list = [x for x in team_list if x != 'NaN']
team_list

['1258',
 '1318',
 '1425',
 '1778',
 '1899',
 '2046',
 '2557',
 '2906',
 '2907',
 '2926',
 '2927',
 '2929',
 '2942',
 '2976',
 '3219',
 '3223',
 '3237',
 '3393',
 '3574',
 '3588',
 '360',
 '3684',
 '3786',
 '4131',
 '4461',
 '4579',
 '4918',
 '492',
 '5468',
 '5937',
 '6350',
 '6443',
 '6503',
 '948',
 '949']

In [17]:
hplot = plt.figure(x_range=team_list, plot_width=800, plot_height=250, title="avg points for teams",
           toolbar_location=None, tools="hover")

hplot.vbar_stack(points, x='index', width=0.4, color=colors, source=data_cds, legend=[" " + x for x in point])

hplot.y_range.start = 0
hplot.x_range.range_padding = 0.1
hplot.xgrid.grid_line_color = None
hplot.axis.minor_tick_line_color = None
hplot.outline_line_color = None
hplot.legend.location = "top_left"
hplot.legend.orientation = "vertical"
hplot.xaxis.major_label_orientation = 3.14/4

bokeh.io.show(hplot)